Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import time
localtime = time.asctime( time.localtime(time.time()) )
print (localtime)

Thu Apr 14 19:58:34 2016


First reload the data we generated in `1_notmnist.ipynb`.

In [14]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [15]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (5000, 784) (5000, 10)
Test set (5000, 784) (5000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(train_labels = train[:, -1]train_labels = train[:, -1]
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 19.533579
Training accuracy: 4.1%
Validation accuracy: 5.6%
Loss at step 100: 2.330626
Training accuracy: 72.3%
Validation accuracy: 69.7%
Loss at step 200: 1.874321
Training accuracy: 75.3%
Validation accuracy: 72.4%
Loss at step 300: 1.617059
Training accuracy: 76.6%
Validation accuracy: 73.2%
Loss at step 400: 1.442316
Training accuracy: 77.4%
Validation accuracy: 73.8%
Loss at step 500: 1.314023
Training accuracy: 78.0%
Validation accuracy: 73.8%
Loss at step 600: 1.214927
Training accuracy: 78.8%
Validation accuracy: 74.1%
Loss at step 700: 1.135430
Training accuracy: 79.2%
Validation accuracy: 74.2%
Loss at step 800: 1.069795
Training accuracy: 79.5%
Validation accuracy: 74.2%
Test accuracy: 82.6%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `sesion.run()`.

In [10]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


ValueError: Cannot feed value of shape (128, 1, 10) for Tensor u'Placeholder_1:0', which has shape '(128, 10)'

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units (nn.relu()) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [17]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  tf_hidden_dataset = tf.placeholder(tf.float32, shape = (batch_size, n_hidden))
    
  

  # Variables.
  weights_0 = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_0 = tf.Variable(tf.zeros([n_hidden]))
    
  weights_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_level = tf.matmul(tf_train_dataset, weights_0) + biases_0
  tf_hidden_dataset = tf.nn.relu(hidden_level)
  logits = tf.matmul(tf_hidden_dataset, weights_1) + biases_1
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  

  def multilayer_perceptron(x, w_0, b_0,w_1, b_1):
        l1 = tf.nn.relu(tf.matmul(x, w_0) + b_0)
        pred = tf.matmul(l1, w_1) + b_1
        return pred
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights_0, biases_0, weights_1, biases_1))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights_0, biases_0, weights_1, biases_1))
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_0) + biases_0), weights_1) + biases_1)
  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_0) + biases_0), weights_1) + biases_1))
  print (r'Run the following model!')

Run the following model!


Let's run it:

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 327.964264
Minibatch accuracy: 10.2%
Validation accuracy: 29.6%
Minibatch loss at step 500: 7.193419
Minibatch accuracy: 89.8%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 1.039628
Minibatch accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 1.002547
Minibatch accuracy: 97.7%
Validation accuracy: 82.1%
Minibatch loss at step 2000: 0.392020
Minibatch accuracy: 96.9%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 0.124046
Minibatch accuracy: 98.4%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 0.113346
Minibatch accuracy: 97.7%
Validation accuracy: 83.0%
Test accuracy: 90.3%


In [37]:
batch_size = 500
n_hidden = [1024, 1024]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #tf_hidden_dataset = tf.placeholder(tf.float32, shape = (batch_size, n_hidden))
    
  

  # Variables.
  #weights_0 = tf.Variable(
  #  tf.truncated_normal([image_size * image_size, n_hidden]))
  weights = [tf.truncated_normal([image_size * image_size, n_hidden[0]]),
             tf.truncated_normal([n_hidden[0], n_hidden[1]]),
             tf.truncated_normal([n_hidden[1], num_labels])]
  #biases_0 = tf.Variable(tf.zeros([n_hidden]))
  #biases_0 = tf.Variable(tf.truncated_normal([n_hidden]))
  biases = [tf.Variable(tf.zeros([n_hidden[0]])),
            tf.Variable(tf.zeros([n_hidden[1]])), 
            tf.Variable(tf.zeros([num_labels]))]  
  #weights_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  #biases_1 = tf.Variable(tf.zeros([num_labels]))
  #biases_1 = tf.Variable(tf.truncated_normal([num_labels]))
  
  # Training computation.
  def multilayer_perceptron(x, w, b):
        h1 = tf.nn.relu(tf.matmul(x, w[0]) + b[0])
        h2 = tf.nn.relu(tf.matmul(h1, w[1]) + b[1])
        pred = tf.matmul(h2, w[2]) + b[2]
        return pred
            
  #hidden_level_1 = tf.matmul(tf_train_dataset, weights_0) + biases_0
  #tf_hidden_dataset = tf.nn.softmax(hidden_level)
  #logits = tf.matmul(tf_hidden_dataset, weights_1) + biases_1,
  logits = multilayer_perceptron(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.000001).minimize(loss)
  
  
  
  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights,biases))
  #valid_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_0) + biases_0), weights_1) + biases_1)
  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_0) + biases_0), weights_1) + biases_1))
  print (r'Run the following model!')

Run the following model!


In [38]:
num_steps =1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  print (type(train_prediction))
  print (type(test_prediction))
  test_pred = test_prediction.eval()
  print (type(test_pred))
  print (test_pred.shape)
  out = np.argmax(test_pred, 1)
  print (out.shape)
  #out = pd.DataFrame(out)
  #out.to_csv('out.csv', header = ['Label'], index_label = range(1,test_pred.shape[0]+1))
  #print ('done!')

Initialized
Minibatch loss at step 0: 6877.421875
Minibatch accuracy: 10.2%
Validation accuracy: 12.3%
Minibatch loss at step 100: 6394.257812
Minibatch accuracy: 14.6%
Validation accuracy: 10.5%
Minibatch loss at step 200: 5721.184082
Minibatch accuracy: 12.2%
Validation accuracy: 10.6%
Minibatch loss at step 300: 6055.330078
Minibatch accuracy: 7.0%
Validation accuracy: 11.7%
Minibatch loss at step 400: 7339.325684
Minibatch accuracy: 8.2%
Validation accuracy: 9.1%
Minibatch loss at step 500: 8203.683594
Minibatch accuracy: 8.4%
Validation accuracy: 14.4%
Minibatch loss at step 600: 8126.503906
Minibatch accuracy: 12.8%
Validation accuracy: 6.7%
Minibatch loss at step 700: 5775.883789
Minibatch accuracy: 6.4%
Validation accuracy: 11.5%
Minibatch loss at step 800: 6654.591797
Minibatch accuracy: 6.6%
Validation accuracy: 10.3%
Minibatch loss at step 900: 5852.946777
Minibatch accuracy: 13.6%
Validation accuracy: 13.8%
Minibatch loss at step 1000: 6630.379883
Minibatch accuracy: 9.8%
V